In [6]:
import pickle

with open("data/19_MM.pickle", "rb") as f:
    data = pickle.load(f)

print(type(data), len(data))
print(data[0].keys())   # 看这一帧的字段


<class 'list'> 34001
dict_keys(['poses3d', 'poses2d', 'activities', 'num_people', 'rgb_frame_number', 'pc', 'raw_fids', 'radar_session_datetime', 'wallclock'])


In [7]:
print(data[0]['activities'])


{'01-PhysicalState': [], '01-Activity': ['undefined'], '01-Objects': [], '01-Notes': [], '02-PhysicalState': [], '02-Activity': [], '02-Objects': []}


In [8]:
import numpy as np

frame_labels = []

for frame in data:
    state = frame['activities'].get('01-PhysicalState', [])
    if state:  
        frame_labels.append(state[0])   # 取第一个标签
    else:
        frame_labels.append("Unknown")  # 没标注的帧

frame_labels = np.array(frame_labels)
print("逐帧标签 shape:", frame_labels.shape)
print("前20个:", frame_labels[:20])


逐帧标签 shape: (34001,)
前20个: ['Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown'
 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown'
 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown' 'Unknown']


In [9]:
samples = np.load("data/poses_19MM_samples.npy")  # (N, 100, 32, 3)
num_samples = samples.shape[0]
window = 100
stride = 50

y = []
for i in range(num_samples):
    start = i * stride
    mid_frame = start + window // 2
    label = frame_labels[mid_frame]
    y.append(label)

y = np.array(y)
np.save("data/y_physicalstate.npy", y)

print("保存成功: data/y_physicalstate.npy, shape:", y.shape)
print("类别分布:", np.unique(y, return_counts=True))


FileNotFoundError: [Errno 2] No such file or directory: 'data/poses_19MM_samples.npy'

In [10]:
for i in [1000, 5000, 10000, 20000]:
    print(i, data[i]['activities']['01-PhysicalState'])


1000 ['a_walk']
5000 []
10000 []
20000 ['p_bent']


In [12]:
import pickle
import numpy as np

# ========= 配置 =========
pickle_file = "data/19_MM.pickle"          # 你的骨架数据
samples_file = "data/poses_19MM.npy"  # 之前保存的样本
window = 100   # 和 make_samples 保持一致
stride = 50    # 和 make_samples 保持一致
save_file = "data/y_physicalstate.npy"     # 保存路径
# ========================

# 1. 加载 pickle 数据
with open(pickle_file, "rb") as f:
    data = pickle.load(f)
print("帧数:", len(data))

# 2. 逐帧提取 PhysicalState
last_label = "Unknown"
frame_labels = []

for frame in data:
    state = frame['activities'].get('01-PhysicalState', [])
    if state:  
        last_label = state[0]   # 更新为当前帧的标签
    frame_labels.append(last_label)

frame_labels = np.array(frame_labels)
print("逐帧标签 shape:", frame_labels.shape)
print("标签种类:", np.unique(frame_labels))

# 3. 加载样本数据
samples = np.load(samples_file)
num_samples = samples.shape[0]
print("样本数:", num_samples)

# 4. 对齐：每个样本用窗口中间帧的标签
y = []
for i in range(num_samples):
    start = i * stride
    mid_frame = start + window // 2
    if mid_frame < len(frame_labels):
        y.append(frame_labels[mid_frame])
    else:
        y.append("Unknown")

y = np.array(y)

# 5. 保存
np.save(save_file, y)
print(f"✅ 已保存标签: {save_file}, shape={y.shape}")
print("类别分布:", dict(zip(*np.unique(y, return_counts=True))))


帧数: 34001
逐帧标签 shape: (34001,)
标签种类: ['Unknown' 'a_walk' 'p_bent' 'p_lie' 'p_sit' 'p_situp' 'p_stand'
 't_bed_turn' 't_bend' 't_lie_to_sit' 't_lie_to_situp' 't_sit_to_lie'
 't_sit_to_stand' 't_situp_to_sit' 't_stand_to_sit' 't_straighten']
样本数: 33955
✅ 已保存标签: data/y_physicalstate.npy, shape=(33955,)
类别分布: {np.str_('Unknown'): np.int64(33276), np.str_('a_walk'): np.int64(144), np.str_('p_bent'): np.int64(27), np.str_('p_lie'): np.int64(42), np.str_('p_sit'): np.int64(202), np.str_('p_situp'): np.int64(5), np.str_('p_stand'): np.int64(187), np.str_('t_bed_turn'): np.int64(10), np.str_('t_bend'): np.int64(17), np.str_('t_lie_to_sit'): np.int64(1), np.str_('t_lie_to_situp'): np.int64(2), np.str_('t_sit_to_lie'): np.int64(3), np.str_('t_sit_to_stand'): np.int64(6), np.str_('t_situp_to_sit'): np.int64(4), np.str_('t_stand_to_sit'): np.int64(8), np.str_('t_straighten'): np.int64(21)}


In [3]:
print(X.shape)
print(y.shape)


(288, 100, 96)
(33955,)


In [4]:
import numpy as np

# 已有数据
# X = (288, 100, 96)
# y = (33955,) 逐帧标签

window = 100
stride = 100   # ⚠️ 你的 X 是怎么切的就用什么 stride
y_samples = []

for start in range(0, len(y) - window + 1, stride):
    end = start + window
    mid = start + window // 2   # 取中间帧
    y_samples.append(y[mid])

y_samples = np.array(y_samples)
print("X shape:", X.shape)
print("y_samples shape:", y_samples.shape)


X shape: (288, 100, 96)
y_samples shape: (339,)


In [8]:
import numpy as np
import pickle

# ================================
# 1. 载入数据
# ================================
with open("data/19_MM.pickle", "rb") as f:
    data = pickle.load(f)

# 提取逐帧标签
frame_labels = []
for frame in data:
    acts = frame.get("activities", {})
    if "01-PhysicalState" in acts and len(acts["01-PhysicalState"]) > 0:
        frame_labels.append(acts["01-PhysicalState"][0])  # 取第一个标签
    else:
        frame_labels.append("Unknown")
frame_labels = np.array(frame_labels)
print("原始逐帧标签 shape:", frame_labels.shape)

# 载入之前保存的骨架数据 (T, 32, 3)
poses_raw = np.load("data/poses_19MM.npy")
print("原始骨架 shape:", poses_raw.shape)

# ================================
# 2. 对齐长度 + 清理全零帧
# ================================
min_len = min(len(frame_labels), len(poses_raw))
frame_labels = frame_labels[:min_len]
poses_raw = poses_raw[:min_len]

mask = ~np.all(poses_raw == 0, axis=(1,2))  # True = 非全零帧
labels_clean = frame_labels[mask]
poses_clean = poses_raw[mask]

print("对齐后帧数:", min_len)
print("清理后标签数:", len(labels_clean))
print("清理后骨架 shape:", poses_clean.shape)

# ================================
# 3. 窗口切分 (生成样本)
# ================================
window = 100   # 窗口长度（帧数）
stride = 100   # 步长
X_samples, y_samples = [], []

for start in range(0, len(poses_clean) - window + 1, stride):
    end = start + window
    mid = start + window // 2   # 取窗口中间帧作为标签
    X_samples.append(poses_clean[start:end])   # shape (100, 32, 3)
    y_samples.append(labels_clean[mid])        # 对应标签

X = np.array(X_samples)   # (num_samples, 100, 32, 3)
y = np.array(y_samples)   # (num_samples, )
print("最终 X shape:", X.shape)
print("最终 y shape:", y.shape)
print("前10个标签:", y[:10])

# ================================
# 4. 保存结果
# ================================
np.save("data/X.npy", X)
np.save("data/y.npy", y)
print("✅ 已保存到 data/X.npy 和 data/y.npy")


原始逐帧标签 shape: (34001,)
原始骨架 shape: (33955, 32, 3)
对齐后帧数: 33955
清理后标签数: 14496
清理后骨架 shape: (14496, 32, 3)
最终 X shape: (144, 100, 32, 3)
最终 y shape: (144,)
前10个标签: ['a_walk' 'a_walk' 'p_stand' 'a_walk' 'a_walk' 'a_walk' 'a_walk' 'a_walk'
 'a_walk' 'a_walk']
✅ 已保存到 data/X.npy 和 data/y.npy


In [9]:
import numpy as np

# 读取刚保存的 X, y
X = np.load("data/X.npy")
y = np.load("data/y.npy")

# reshape: (144, 100, 32, 3) -> (144, 100, 96)
X_lstm = X.reshape(X.shape[0], X.shape[1], -1)

print("LSTM 输入 shape:", X_lstm.shape)
print("y shape:", y.shape)

# 保存结果
np.save("data/X_lstm.npy", X_lstm)
np.save("data/y_lstm.npy", y)
print("✅ 已保存 data/X_lstm.npy 和 data/y_lstm.npy")


LSTM 输入 shape: (144, 100, 96)
y shape: (144,)
✅ 已保存 data/X_lstm.npy 和 data/y_lstm.npy


In [11]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# ==============================
# 1. 加载数据
# ==============================
X = np.load("data/X_lstm.npy")   # (N, 100, 96)
y = np.load("data/y_lstm.npy")   # (N, )

print("X shape:", X.shape)
print("y shape:", y.shape)

# 标签编码
le = LabelEncoder()
y_encoded = le.fit_transform(y)
num_classes = len(le.classes_)
print("类别数:", num_classes, "类别:", le.classes_)

X_train, X_val, y_train, y_val = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=None  # ❌ 不分层
)


# 转换为 tensor
X_train = torch.tensor(X_train, dtype=torch.float32)
X_val   = torch.tensor(X_val, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_val   = torch.tensor(y_val, dtype=torch.long)

# ==============================
# 2. 定义 LSTM 模型
# ==============================
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim=96, hidden_dim=128, num_layers=2, num_classes=10):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers,
                            batch_first=True, dropout=0.3)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)  # out: (batch, seq, hidden)
        out = out[:, -1, :]    # 取最后一个时间步
        out = self.fc(out)
        return out

model = LSTMClassifier(input_dim=96, hidden_dim=128,
                       num_layers=2, num_classes=num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# ==============================
# 3. 训练循环
# ==============================
epochs = 20
batch_size = 16

for epoch in range(epochs):
    # ---- Train ----
    model.train()
    perm = torch.randperm(X_train.size(0))
    total_loss, correct = 0, 0

    for i in range(0, X_train.size(0), batch_size):
        idx = perm[i:i+batch_size]
        xb, yb = X_train[idx], y_train[idx]

        optimizer.zero_grad()
        outputs = model(xb)
        loss = criterion(outputs, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (outputs.argmax(1) == yb).sum().item()

    train_acc = correct / X_train.size(0)

    # ---- Validation ----
    model.eval()
    with torch.no_grad():
        outputs = model(X_val)
        val_loss = criterion(outputs, y_val).item()
        val_acc = (outputs.argmax(1) == y_val).sum().item() / X_val.size(0)

    print(f"Epoch [{epoch+1}/{epochs}] "
          f"Train Loss: {total_loss:.4f} | Train Acc: {train_acc:.4f} "
          f"| Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

# ==============================
# 4. 评估
# ==============================
model.eval()
with torch.no_grad():
    y_pred = model(X_val).argmax(1).cpu().numpy()
    y_true = y_val.cpu().numpy()

print("\n分类报告:")
print(classification_report(y_true, y_pred, target_names=le.classes_))

print("混淆矩阵:")
print(confusion_matrix(y_true, y_pred))

# ==============================
# 5. 保存模型和标签映射
# ==============================
torch.save(model.state_dict(), "data/lstm_baseline.pth")
np.save("data/label_classes.npy", le.classes_)
print("✅ 模型和标签映射已保存")


X shape: (144, 100, 96)
y shape: (144,)
类别数: 10 类别: ['Unknown' 'a_walk' 'p_bent' 'p_lie' 'p_sit' 'p_stand' 't_bed_turn'
 't_bend' 't_stand_to_sit' 't_straighten']
Epoch [1/20] Train Loss: 17.3454 | Train Acc: 0.2000 | Val Loss: 2.0719 | Val Acc: 0.3103
Epoch [2/20] Train Loss: 15.5080 | Train Acc: 0.3130 | Val Loss: 2.0957 | Val Acc: 0.3103
Epoch [3/20] Train Loss: 14.5083 | Train Acc: 0.3130 | Val Loss: 2.0472 | Val Acc: 0.3103
Epoch [4/20] Train Loss: 15.2651 | Train Acc: 0.3130 | Val Loss: 1.9990 | Val Acc: 0.3103
Epoch [5/20] Train Loss: 15.0481 | Train Acc: 0.3304 | Val Loss: 2.0225 | Val Acc: 0.2414
Epoch [6/20] Train Loss: 14.3531 | Train Acc: 0.3217 | Val Loss: 2.0384 | Val Acc: 0.3103
Epoch [7/20] Train Loss: 14.0567 | Train Acc: 0.3130 | Val Loss: 2.0278 | Val Acc: 0.3103
Epoch [8/20] Train Loss: 13.9372 | Train Acc: 0.3130 | Val Loss: 2.0541 | Val Acc: 0.3103
Epoch [9/20] Train Loss: 14.8840 | Train Acc: 0.3130 | Val Loss: 2.0223 | Val Acc: 0.2759
Epoch [10/20] Train Loss: 1

ValueError: Number of classes, 8, does not match size of target_names, 10. Try specifying the labels parameter

In [12]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# 确保 y_true / y_pred 都是 numpy 数组
y_true = y_val.cpu().numpy()
y_pred = np.array(y_pred)

# 找到验证集中实际出现过的类别
unique_labels = np.unique(y_true)

print("\n分类报告:")
print(classification_report(
    y_true, y_pred,
    labels=unique_labels,
    target_names=le.classes_[unique_labels]
))

print("混淆矩阵:")
print(confusion_matrix(y_true, y_pred, labels=unique_labels))



分类报告:
                precision    recall  f1-score   support

       Unknown       0.00      0.00      0.00         5
        a_walk       0.00      0.00      0.00         7
        p_bent       0.00      0.00      0.00         2
         p_lie       0.00      0.00      0.00         2
         p_sit       0.00      0.00      0.00         2
       p_stand       0.31      1.00      0.47         9
t_stand_to_sit       0.00      0.00      0.00         1
  t_straighten       0.00      0.00      0.00         1

      accuracy                           0.31        29
     macro avg       0.04      0.12      0.06        29
  weighted avg       0.10      0.31      0.15        29

混淆矩阵:
[[0 0 0 0 0 5 0 0]
 [0 0 0 0 0 7 0 0]
 [0 0 0 0 0 2 0 0]
 [0 0 0 0 0 2 0 0]
 [0 0 0 0 0 2 0 0]
 [0 0 0 0 0 9 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 0]]


D:\Conda\miniconda3\envs\panoptes_pc_hpe\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
D:\Conda\miniconda3\envs\panoptes_pc_hpe\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
D:\Conda\miniconda3\envs\panoptes_pc_hpe\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is

In [13]:
import numpy as np
y = np.load("data/y_lstm.npy", allow_pickle=True)
unique, counts = np.unique(y, return_counts=True)
print(dict(zip(unique, counts)))


{np.str_('Unknown'): np.int64(17), np.str_('a_walk'): np.int64(28), np.str_('p_bent'): np.int64(6), np.str_('p_lie'): np.int64(6), np.str_('p_sit'): np.int64(27), np.str_('p_stand'): np.int64(45), np.str_('t_bed_turn'): np.int64(3), np.str_('t_bend'): np.int64(7), np.str_('t_stand_to_sit'): np.int64(1), np.str_('t_straighten'): np.int64(4)}
